## Getting started with ENOT

This notebook describes the basic steps you need to optimize an architecture.

### Notebook consists of next main stages:
1. Setup the environment
1. Prepare dataset and create dataloaders
1. Create the model and move it to search space
1. Pretrain constructed search space
1. Search best architecture
1. Tune model with best architecture

## 1. Setup the environment
First, let's set up the environment and common imports.

In [ ]:
# Copy your license file to $HOME/.enot/enot.lic or set full path to licence file 
# through environment variable ENOT_LIC_FILE
#
# Important note: no quotes
# %env ENOT_LIC_FILE=/FULL/PATH/TO/your_company.lic

In [ ]:
import os

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
# You should change to free GPU
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
from pathlib import Path

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_optimizer import RAdam

from enot.models import SearchSpaceModel
from enot.models.mobilenet import build_mobilenet
from enot.phases import pretrain
from enot.phases import search
from enot.phases import train

from enot_utils.metric_utils import accuracy
from enot_utils.schedulers import WarmupScheduler

from tutorial_utils.checkpoints import download_getting_started_pretrain_checkpoint
from tutorial_utils.dataset import create_imagenette_dataloaders

### In the next cell we setup all required dirs

* `ENOT_HOME_DIR` - is root dir for all other dirs
* `ENOT_DATASETS_DIR` - is root dir for datasets (imagenette2)
* `PROJECT_DIR` - is root dir for output data (checkpoints, logs...) of current tutorial

In [ ]:
ENOT_HOME_DIR = Path.home() / '.enot'
ENOT_DATASETS_DIR = ENOT_HOME_DIR / 'datasets'
PROJECT_DIR = ENOT_HOME_DIR / 'getting_started'

ENOT_HOME_DIR.mkdir(exist_ok=True)
ENOT_DATASETS_DIR.mkdir(exist_ok=True)
PROJECT_DIR.mkdir(exist_ok=True)

## 2. Prepare dataset and create dataloaders

In this example we will use dataset Imagenette2. <br>
`create_imagenette_dataloaders` function do all dirty work: 
1. automatically downloads and unpack dataset in `ENOT_DATASETS_DIR`
1. split dataset on 4 parts and save annotation of every part in `PROJECT_DIR`:
    * train - for pretrain and tuning stages (`PROJECT_DIR`/train.csv)
    * validation - to choose checkpoint for architecture optimization (`PROJECT_DIR`/validation.csv)
    * search - for architecture optimization stage (`PROJECT_DIR`/search.csv)
    * test - held-out data for testing (`PROJECT_DIR`/test.csv)
1. create dataloaders for every stage and returns dataloaders as dictionary:
    * pretrain_train_dataloader - pretrain train dataloader (based on `PROJECT_DIR`/train.csv)
    * pretrain_validation_dataloader - pretrain validation dataloader (based on `PROJECT_DIR`/validation.csv)
    * search_train_dataloader - search train dataloader (based on `PROJECT_DIR`/search.csv)
    * search_validation_dataloader - search validation dataloader (based on `PROJECT_DIR`/search.csv)
    * tune_train_dataloader - tune train dataloader (the same as pretrain_train_dataloader)
    * tune_validation_dataloader - tune validation dataloader (the same as pretrain_validation_dataloader)

**NOTE:**<br>
CSV annotations follow this format:
```
filepath,label
<relative_path_1>,<int_label_1>
<relative_path_2>,<int_label_2>
...
```

In [ ]:
dataloaders = create_imagenette_dataloaders(
    dataset_root_dir=ENOT_DATASETS_DIR, 
    project_dir=PROJECT_DIR,
    input_size=(224, 224),
    batch_size=32,
)

## 3. Create the model and move it to search space

Our architecture optimization procedure selects the best combination of operations from a user-defined search space. The easiest way to define one is to take a base architecture and add similar operations with different parameters: kernel size, expansion ratio, etc. You can also add different kinds of operations or implement your own (see "Tutorial - adding a custom operation" for this).

In this example we take MobileNet v2 as a base architecture and make a search space from MobileNet inverted bottleneck blocks.

First, let's define a model for image classification tasks. MobileNet like models can be built by `build_mobilenet` function. `build_mobilenet` function returns model with the following structure: (inputs) -> stem -> blocks -> head -> (outputs). Stem and head have a fixed structure, blocks consist of user-defined operations to choose from at the search stage. The template class will follow MobileNet v2 structure, [as seen in torchvision](https://github.com/pytorch/vision/blob/master/torchvision/models/mobilenet.py).

In [ ]:
# The search space will have these ops to choose from in each layer
# Short format for operations is 'Name_param1=value1_param2=value2...'
# MIB is MNv2 inverted bottleneck. k is kernel size, t is expansion ratio
# See more in-depth info in "Tutorial - adding custom operations"
SEARCH_OPS = [
    'MIB_k=3_t=6',
    'MIB_k=5_t=6',
    'MIB_k=7_t=6',
]

# build model
model = build_mobilenet(
    search_ops=SEARCH_OPS,
    num_classes=10,
    blocks_out_channels=[24, 32, 64, 96, 160, 320],
    blocks_count=[2, 2, 2, 1, 2, 1],
    blocks_stride=[2, 2, 2, 1, 2, 1],
)
# move model to search space
search_space = SearchSpaceModel(model, train_loader=dataloaders['pretrain_train_dataloader']).cuda()

## 4. Pretrain constructed search space

We've implemented a default training procedure for you. You only need to set up search space, data loaders, transforms, and optimizer and run it.

**IMPORTANT:**<br>
`N_EPOCHS` should be in range >= 300, if you wanna get good pretrain. In this tutorial we set `N_EPOCHS` = 3 and download checkpoint of trained model from Google Drive. 

In [ ]:
# define directory for text logs and tensorboard logs
pretrain_dir = PROJECT_DIR / 'pretrain'
pretrain_dir.mkdir(exist_ok=True)

N_EPOCHS = 1
N_WARMUP_EPOCHS = 1
len_train = len(dataloaders['pretrain_train_dataloader'])

optimizer = SGD(params=search_space.model_parameters(), lr=0.06, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=len_train*N_EPOCHS, eta_min=1e-8)
scheduler = WarmupScheduler(scheduler, warmup_steps=len_train*N_WARMUP_EPOCHS)
loss_function = nn.CrossEntropyLoss().cuda()

pretrain(
    search_space=search_space,
    exp_dir=pretrain_dir,
    train_loader=dataloaders['pretrain_train_dataloader'],
    valid_loader=dataloaders['pretrain_validation_dataloader'],
    optimizer=optimizer,
    scheduler=scheduler,
    metric_function=accuracy,
    loss_function=loss_function,
    epochs=N_EPOCHS,
)

In [ ]:
# You can find best pretrain checkpoint and load it into your search space.
checkpoint_path = pretrain_dir / 'getting_started_pretrain_checkpoint.pth'
download_getting_started_pretrain_checkpoint(checkpoint_path)

search_space.load_state_dict(
    torch.load(checkpoint_path)['model'],
)

## 5. Search best architecture
Now that you have a trained search space you can run the search phase. The setup is similar to pretrain.

In [ ]:
# define directory for text logs and tensorboard logs
search_dir = PROJECT_DIR / 'search'
search_dir.mkdir(exist_ok=True)

optimizer = RAdam(search_space.architecture_parameters(), lr=0.01)

search(
    search_space=search_space,
    exp_dir=search_dir,
    search_loader=dataloaders['search_train_dataloader'],
    valid_loader=dataloaders['search_validation_dataloader'],
    optimizer=optimizer,
    loss_function=loss_function,
    metric_function=accuracy,
    latency_loss_weight=2.0e-3,
    epochs=5,
)

## 6. Tune model with best architecture
Now we take the best architecture in search space and create a regular model using it, then we run finetune procedure.

In [ ]:
# get regular model with best architecture
best_model = search_space.get_network_with_best_arch().cuda()

In [ ]:
# define directory for text logs and tensorboard logs
tune_dir = PROJECT_DIR / 'tune'
tune_dir.mkdir(exist_ok=True)

optimizer = RAdam(best_model.parameters(), lr=5e-3, weight_decay=4e-5)

train(
    model=best_model,
    exp_dir=tune_dir,
    train_loader=dataloaders['tune_train_dataloader'],
    valid_loader=dataloaders['tune_validation_dataloader'],
    optimizer=optimizer,
    loss_function=loss_function,
    metric_function=accuracy,
    epochs=5,
)